In [4]:
# Importing libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [5]:
# Creating path for df
path = r"/content/drive/MyDrive/Pythondataanalytics/Instacartanalysis"

Creating bar chart

In [6]:
# Importing customers data set,
customers = pd.read_csv (os.path.join(path, '2) Data', 'Original data', 'customers.csv'), index_col = False)

In [7]:
# Checking imported data,
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
customers.shape

(206209, 10)

Q4) Wrangle the data so that it follows consistent logic; for example, rename columns with illogical names and drop columns that dont add anything to analysis.

In [9]:
# Looking at 'date_joined' column to see if all dates are the same,
customers['date_joined'].value_counts(dropna = False)

9/17/2018     213
2/10/2018     212
4/1/2019      211
9/21/2019     211
12/19/2017    210
             ... 
9/1/2018      141
1/22/2018     140
11/24/2017    139
7/18/2019     138
8/6/2018      128
Name: date_joined, Length: 1187, dtype: int64

Variations in 'date_joined,' so I will not consider this "place-holder" data and leave it as it is

In [10]:
#Renaming 'First Name' column to 'First_Name',
customers.rename(columns = {'First Name' : 'First_Name'}, inplace = True)

In [11]:
# Renaming 'Surnam' column to 'Last_Name',
customers.rename(columns = {'Surnam' : 'Last_Name'}, inplace = True)

In [12]:
# Renaming 'STATE' column to 'State',
customers.rename(columns = {'STATE' : 'State'}, inplace = True)

In [13]:
# Renaming 'n_dependants' column to 'number_of_dependants',
customers.rename(columns = {'n_dependants' : 'number_of_dependants'}, inplace = True)

In [14]:
# Checking output for column name changes,
customers.head()

,user_id,First_Name,Last_Name,Gender,State,Age,date_joined,number_of_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


Q5) Complete the fundamental data quality and consistency checks you’ve learned throughout this Achievement; for example, check for and address missing values and duplicates, and convert any mixed-type data

In [15]:
# Using .describe() to look for inconsistencies,
customers.describe()

,user_id,Age,number_of_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


There is no need for 'user_id' to be an integer. This isn't a column that will need mathematical calculations done to it. So, I will change the data type to a string. All stats for these other columns seem reasonable. Although, it is a little surprising the max number of dependents is 3. It seems there would be at least a few families with more than 3 children or other dependents.

In [16]:
customers['user_id'] = customers['user_id'].astype('str')

In [17]:
# Checking the data type for 'user_id' has changed and checking data types for all other columns,
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_id               206209 non-null  object
 1   First_Name            194950 non-null  object
 2   Last_Name             206209 non-null  object
 3   Gender                206209 non-null  object
 4   State                 206209 non-null  object
 5   Age                   206209 non-null  int64 
 6   date_joined           206209 non-null  object
 7   number_of_dependants  206209 non-null  int64 
 8   fam_status            206209 non-null  object
 9   income                206209 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 15.7+ MB


Maximum number for a client's age is 81, and the maximum number of dependents is 3, I will change both of these to int8 data types to save space. For the same reason, I'll also change the 'income' column to int32

In [18]:
customers['Age'] = customers['Age'].astype('int8')

In [19]:
customers['number_of_dependants'] = customers['number_of_dependants'].astype('int8')

In [20]:
customers['income'] = customers['income'].astype('int32')

In [21]:
# Checking data type changes,
customers.dtypes

user_id                 object
First_Name              object
Last_Name               object
Gender                  object
State                   object
Age                       int8
date_joined             object
number_of_dependants      int8
fam_status              object
income                   int32
dtype: object

#Looking for mixed data types

In [22]:
# looking in customers df to see if there are any mixed type,
for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (customers[weird]) > 0:
        print (col)

First_Name


In [23]:
# The 'First_Name' column has mixed data types. These should be strings, so I will set the whole column to a string data type,
customers['First_Name'] = customers['First_Name'].astype('str')

In [24]:
# Checking data type change,
customers['First_Name'].dtype

dtype('O')

In [25]:
# Checking again for mixed data types,
for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (customers[weird]) > 0:
        print (col)

There are no more mixed data type columns and lets look for missing values.
   

In [26]:
# Finding missing values and summing them,
customers.isnull().sum()

user_id                 0
First_Name              0
Last_Name               0
Gender                  0
State                   0
Age                     0
date_joined             0
number_of_dependants    0
fam_status              0
income                  0
dtype: int64

No missing values in any column and lets look for dupplicates.

In [27]:
# Looking for duplicate records,
customers[customers.duplicated()]

,user_id,First_Name,Last_Name,Gender,State,Age,date_joined,number_of_dependants,fam_status,income


No duplicate records in this data set

In [28]:
# Making sure the data frame shape is the same after the consistency checks (since no records were excluded),
customers.shape

(206209, 10)

Q6) Combine your customer data with the rest of your prepared Instacart data. (Hint: Make sure the key columns are the same data type!)

In [29]:
# Importing data set to merge with customers data set,
ords_prods_merge = pd.read_pickle(os.path.join(path, '2) Data', 'Prepared data', 'orders_products_merged_updated_2.0.pkl'))

In [30]:
# Checking data set import,
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_orders,order_frequency_flag
0,198637,175298,prior,9,0,16,3.0,39812,23,0,...,Low-range product,Busiest day,Busiest days,Most orders,38,Regular customer,6.188235,Low spender,6.0,Frequent customer
1,9357,6501,prior,22,6,21,21.0,39812,7,1,...,Low-range product,Regularly busy,Regularly busy,Average orders,28,Regular customer,7.775000,Low spender,11.5,Regular customer
2,1599735,180044,prior,9,6,13,22.0,39812,4,0,...,Low-range product,Regularly busy,Regularly busy,Most orders,12,Regular customer,4.885714,Low spender,22.0,Non-frequent customer
3,227255,12074,prior,26,4,23,3.0,39812,23,1,...,Low-range product,Least busy,Least busy days,Fewest orders,30,Regular customer,7.761111,Low spender,21.0,Non-frequent customer
4,1675773,192641,prior,11,0,12,30.0,39812,1,1,...,Low-range product,Busiest day,Busiest days,Most orders,11,Regular customer,3.700000,Low spender,30.0,Non-frequent customer


In [31]:
# Checking data type for 'user_id' column,
ords_prods_merge['user_id'].dtype

dtype('int64')

In [32]:
# Changing the 'user_id' column to a string data type to match the data type for that same column in the customers data frame,
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')

In [33]:
# Rechecking data type for 'user_id',
ords_prods_merge['user_id'].dtype

dtype('O')

Remove the '_merge' column from the 'ords_prods_merge' data frame before merging it with 'customers' data frame

In [34]:
# Dropping '_merge' column from ords_prods_merge data frame,
ords_prods_merge = ords_prods_merge.drop(columns = ['_merge'])

In [35]:
# Using an inner join to merge the 'customers' data frame with the 'ords_prods_merge' data frame,
ords_prods_custs_merge = ords_prods_merge.merge(customers, on = 'user_id', indicator = True)

In [36]:
# Checking final output,
ords_prods_custs_merge.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,...,First_Name,Last_Name,Gender,State,Age,date_joined,number_of_dependants,fam_status,income,_merge
0,198637,175298,prior,9,0,16,3.0,39812,23,0,...,Diane,Boyer,Female,Georgia,47,2/25/2018,3,married,166223,both
1,3210430,175298,prior,10,5,15,5.0,47209,11,1,...,Diane,Boyer,Female,Georgia,47,2/25/2018,3,married,166223,both
2,1111015,175298,prior,11,0,17,2.0,3957,7,1,...,Diane,Boyer,Female,Georgia,47,2/25/2018,3,married,166223,both
3,1423966,175298,prior,17,0,17,7.0,3957,6,1,...,Diane,Boyer,Female,Georgia,47,2/25/2018,3,married,166223,both
4,2333772,175298,prior,29,6,17,9.0,47672,12,0,...,Diane,Boyer,Female,Georgia,47,2/25/2018,3,married,166223,both


In [37]:
ords_prods_custs_merge['_merge'].value_counts()

both          999993
left_only          0
right_only         0
Name: _merge, dtype: int64

In [38]:
# Checking inner merge by looking at outer merge,
merge_check = ords_prods_merge.merge(customers, on = 'user_id', indicator = True, how = 'outer')

In [39]:
merge_check['_merge'].value_counts()

both          999993
right_only     36439
left_only          0
Name: _merge, dtype: int64

In [40]:
ords_prods_custs_merge.to_pickle(os.path.join(path, '2) Data', 'Prepared data', 'ords_prods_custs_merge.pkl'))